In [1]:
import pandas as pd
import os
import re

folder_path = "../data/external/features"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

dataframes = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)

    start = file.find("indicator_") + len("indicator_")
    end = file.rfind(".csv")
    substring = file[start:end].rstrip("_")

    df["Indicator"] = substring
    df["Filename"] = file
    df = df[["DateTime", "Close", "Frequency", "Indicator", "Filename"]].copy()
    dataframes.append(df)
    del df

df = pd.read_csv('../data/external/inflation_target/historical_country_United_States_indicator_inflation_rate.csv')
df.rename(columns={"Value" : "Close"}, inplace=True)
df["Indicator"] = "inflation_rate"
df["Filename"] = "historical_country_United_States_indicator_inflation_rate.csv"
df = df[["DateTime", "Close", "Frequency", "Indicator", "Filename"]].copy()
dataframes.append(df)
del df 

full_data = pd.concat(dataframes, ignore_index=True)
full_data['DateTime'] = pd.to_datetime(full_data['DateTime'])
full_data['DateTime'] = full_data['DateTime'].dt.strftime('%Y-%m-%d')
full_data.rename(columns={"DateTime" : "Date"}, inplace=True)

/var/folders/c6/rtc3mpzn52n42dpmym41ynbr0000gn/T/ipykernel_21598/400152105.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_data = pd.concat(dataframes, ignore_index=True)


In [2]:
print(f"Количество файлов: {len(set(csv_files))}.")
print(f"Количество загруженных csv файлов: {len(dataframes) - 1}.")
print(f"Данные загружены из {len(full_data.Filename.unique())} файлов.")

if len(full_data.Filename.unique()) <= len(set(csv_files)):
    print(f"При загрузке пропущены данные из файла: {set(csv_files) - set(full_data.Filename.unique())}")

Количество файлов: 299.
Количество загруженных csv файлов: 299.
Данные загружены из 299 файлов.
При загрузке пропущены данные из файла: {'historical_country_united_states_indicator_credit_rating_.csv'}


In [3]:
for name in csv_files:
    if "inflation" in name:
        print(name)

historical_country_united_states_indicator_inflation_expectations_.csv
historical_country_united_states_indicator_energy_inflation_.csv
historical_country_united_states_indicator_michigan_inflation_expectations_.csv
historical_country_united_states_indicator_services_inflation_.csv
historical_country_united_states_indicator_michigan_5_year_inflation_expectations_.csv
historical_country_united_states_indicator_rent_inflation_.csv
historical_country_united_states_indicator_food_inflation_.csv
historical_country_united_states_indicator_core_inflation_rate_.csv
historical_country_united_states_indicator_core_inflation_rate_mom_.csv
historical_country_united_states_indicator_producer_price_inflation_mom_.csv
historical_country_united_states_indicator_inflation_rate_mom_.csv


In [4]:
print(full_data.shape, end="\n\n")
print(full_data.dtypes, end="\n\n")
print(full_data.describe, end="\n\n")
print(full_data.isna().sum())

(246554, 5)

Date          object
Close        float64
Frequency     object
Indicator     object
Filename      object
dtype: object

<bound method NDFrame.describe of               Date   Close Frequency       Indicator  \
0       2000-12-31  4335.0   Monthly   job_vacancies   
1       2001-01-31  5592.0   Monthly   job_vacancies   
2       2001-02-28  4844.0   Monthly   job_vacancies   
3       2001-03-31  4514.0   Monthly   job_vacancies   
4       2001-04-30  5095.0   Monthly   job_vacancies   
...            ...     ...       ...             ...   
246549  2024-11-30     2.7   Monthly  inflation_rate   
246550  2024-12-31     2.9   Monthly  inflation_rate   
246551  2025-01-31     3.0   Monthly  inflation_rate   
246552  2025-02-28     2.8   Monthly  inflation_rate   
246553  2025-03-31     2.4   Monthly  inflation_rate   

                                                 Filename  
0       historical_country_united_states_indicator_job...  
1       historical_country_united_states

In [5]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246554 entries, 0 to 246553
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Date       246554 non-null  object 
 1   Close      246554 non-null  float64
 2   Frequency  246554 non-null  object 
 3   Indicator  246554 non-null  object 
 4   Filename   246554 non-null  object 
dtypes: float64(1), object(4)
memory usage: 9.4+ MB


In [6]:
full_data.head()

,Date,Close,Frequency,Indicator,Filename
0,2000-12-31,4335.0,Monthly,job_vacancies,historical_country_united_states_indicator_job...
1,2001-01-31,5592.0,Monthly,job_vacancies,historical_country_united_states_indicator_job...
2,2001-02-28,4844.0,Monthly,job_vacancies,historical_country_united_states_indicator_job...
3,2001-03-31,4514.0,Monthly,job_vacancies,historical_country_united_states_indicator_job...
4,2001-04-30,5095.0,Monthly,job_vacancies,historical_country_united_states_indicator_job...


In [7]:
full_data.Frequency.value_counts()

Frequency
Monthly      109130
Weekly        72735
Daily         44411
Quarterly      9957
weekly         8880
Yearly         1441
Name: count, dtype: int64

In [8]:
full_data = full_data[full_data.Frequency != "Yearly"].copy()

In [9]:
full_data.to_csv("../data/interim/full_collected_data.csv", index=False)